In [21]:
# Get data here: https://datahack.analyticsvidhya.com/contest/practice-problem-big-mart-sales-iii/
import pandas as pd
import numpy as np
from rgf.sklearn import RGFRegressor
from sklearn import preprocessing 
from sklearn.metrics import mean_squared_error, r2_score  # for regression labels
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

In [2]:
train = pd.read_csv("Big_Mart_Train.csv")
test = pd.read_csv("Big_Mart_Test.csv")

In [3]:
train.Item_Weight = train.Item_Weight.fillna(np.nanmedian(train.Item_Weight))
test.Item_Weight = test.Item_Weight.fillna(np.nanmedian(test.Item_Weight))

In [4]:
train.Outlet_Size = train.Outlet_Size.fillna(train.Outlet_Size.mode().iloc[0])
test.Outlet_Size = test.Outlet_Size.fillna(train.Outlet_Size.mode().iloc[0])

In [5]:
train.Item_Fat_Content = train.Item_Fat_Content.replace(['low fat', 'LF'], ['Low Fat', 'Low Fat'])
test.Item_Fat_Content = test.Item_Fat_Content.replace(['low fat', 'LF'], ['Low Fat', 'Low Fat'])
train.Item_Fat_Content = train.Item_Fat_Content.replace(['reg'], ['Regular'])
test.Item_Fat_Content = test.Item_Fat_Content.replace(['reg'], ['Regular'])

In [6]:
train.Outlet_Establishment_Year = 2017 - train.Outlet_Establishment_Year
test.Outlet_Establishment_Year = 2017 - test.Outlet_Establishment_Year

In [7]:
# label encoding, do this by combining train and test together

test['Item_Outlet_Sales'] = 0
combi = train.append(test)
number = LabelEncoder()

for i in combi.columns:
    if (combi[i].dtype == 'object'):
        combi[i] = number.fit_transform(combi[i].astype('str'))
        combi[i] = combi[i].astype('object')
        
train = combi[:train.shape[0]]
test = combi[train.shape[0]:]

In [8]:
test.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,1114,20.750,0,0.007565,13,107.8622,9,18,1,0,1,0.0
1,1078,8.300,1,0.038428,4,87.3198,2,10,1,1,1,0.0
2,1420,14.600,0,0.099575,11,241.7538,0,19,1,2,0,0.0
3,817,7.315,0,0.015388,13,155.0340,2,10,1,1,1,0.0
4,1197,12.500,1,0.118599,4,234.2300,5,32,1,2,3,0.0


In [9]:
from sklearn.model_selection import train_test_split

train = train.drop('Item_Identifier',axis=1)
test = test.drop('Item_Identifier',axis=1)

y = train.Item_Outlet_Sales
X = train.drop(['Item_Outlet_Sales'], axis=1)

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=410)

In [10]:
X_train.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
2438,8.51,1,0.078339,13,172.9422,1,30,0,2,1
5523,12.60,0,0.087437,6,109.9228,6,13,2,1,1
1097,12.60,1,0.040163,13,181.1660,5,32,1,2,3
3146,8.52,0,0.045292,14,153.5682,0,19,1,2,0
2582,20.35,0,0.090480,13,119.5466,0,19,1,2,0


In [11]:
param_set = {'max_leaf':[1000,1200,1300,1400,1500,1600,1700,1800,1900,2000],
              'l2':[0.1,0.2,0.3],
              'min_samples_leaf':[4,10]}

gsearch = GridSearchCV(estimator = RGFRegressor(learning_rate =0.1, loss='Log'), 
 param_grid = param_set,n_jobs=7, cv=10, scoring='neg_mean_squared_error')

In [13]:
rgf_model = gsearch.fit(X_train, y_train)
rgf_model.cv_results_, rgf_model.best_params_, rgf_model.best_score_

({'mean_fit_time': array([ 0.3684557 ,  0.59250512,  0.45376027,  0.44017498,  0.45626903,
          0.44970973,  0.43894658,  0.42482817,  0.49904621,  0.43536808,
          0.44014027,  0.44622054,  0.43466704,  0.45483651,  0.43847854,
          0.45948772,  0.44655688,  0.45149384,  0.45311253,  0.46100452,
          0.44782608,  0.45524945,  0.4535507 ,  0.47965879,  0.47501144,
          0.45171614,  0.47223487,  0.46248789,  0.45438995,  0.46145663,
          0.46803682,  0.442698  ,  0.44157901,  0.43842547,  0.44351537,
          0.44436464,  0.4477586 ,  0.44880841,  0.4553823 ,  0.4346102 ,
          0.44632061,  0.44164464,  0.45784819,  0.4479074 ,  0.45300131,
          0.44986243,  0.43918016,  0.48512106,  0.471105  ,  0.46990399,
          0.45762208,  0.44246798,  0.45598526,  0.4642153 ,  0.44291341,
          0.43375523,  0.44405489,  0.44084697,  0.45812194,  0.43781297]),
  'mean_score_time': array([ 0.04046104,  0.0551976 ,  0.04598184,  0.04738538,  0.0429038 ,


In [15]:
y_pred = rgf_model.predict(X_validation)
print(y_pred)

[ 2173.45  2173.45  2173.45 ...,  2173.45  2173.45  2173.45]


In [22]:
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_validation, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_validation, y_pred))

Mean squared error: 3041420.35
Variance score: -0.00
